In [1]:
import pandas as pd
import re

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("../data/nips_cleaned.csv")

In [4]:
df.columns

Index(['title', 'abstract', 'pdf_link', 'year', 'track', 'track_original',
       'main_track', 'text'],
      dtype='object')

In [6]:
df = df[['text', 'main_track']]

In [7]:
labels = df.main_track.unique().tolist()

In [8]:
df = pd.get_dummies(df, columns = ['main_track'])

In [9]:
df = df.rename(columns=lambda x: re.sub('main_track_','',x))

In [10]:
#prep the data

In [11]:
df.shape

(7882, 9)

In [17]:
import random
random.seed(30)

train_size = 0.6*7916

train_indices = random.sample(range(0, 7882), int(0.6*7916))

In [18]:
df_train = df.loc[train_indices].reset_index(drop=True)

In [19]:
df_val = df.drop(train_indices, axis=0).reset_index(drop=True)

In [20]:
df_train.to_csv("../data/train.csv", index=False)
df_val.to_csv("../data/val.csv", index=False)

In [21]:
df_val.shape

(3133, 9)

In [22]:
df_train.shape

(4749, 9)

In [37]:
df_train.head()

,text,Algorithms,Applications,"Data, Challenges, Implementations, and Software",Deep Learning,Neuroscience and Cognitive Science,Optimization,Probabilistic Methods,Reinforcement Learning and Planning,Theory
0,deep structured models are widely used for tas...,0,0,0,1,0,0,0,0,0
1,recurrent neural networks have achieved state...,0,1,0,0,0,0,0,0,0
2,optimizing task-related mathematical model is ...,0,1,0,0,0,0,0,0,0
3,the dueling bandit is learning framework where...,0,0,0,0,0,1,0,0,0
4,in this paper we consider the problem of how r...,0,0,0,0,0,0,0,1,0


In [23]:
f = open("labels.csv", "a")
for item in labels:
    f.write(item)
    f.write("\n")
f.close()

In [24]:
import torch
import apex

#from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

from fast_bert.data import BertDataBunch
from fast_bert import BertLearner
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy

ModuleNotFoundError: No module named 'torch'

In [61]:
DATA_PATH = "C:/Users/tneha/Desktop/csetRA/conferences/conference-papers/data/nips/bert/"

LABEL_PATH = "C:/Users/tneha/Desktop/csetRA/conferences/conference-papers/data/nips/bert/"

OUTPUT_DIR = "C:/Users/tneha/Desktop/csetRA/conferences/conference-papers/data/nips/bert/"

In [62]:
args = {
    "max_seq_length": 512,
    "do_lower_case": True,
    "train_batch_size": 32,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

In [63]:
#tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_PATH, do_lower_case=args['do_lower_case'])

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [65]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='bert-base-uncased',
                          train_file='train.csv',
                          val_file='val.csv',
                          label_file='labels.csv',
                          text_col='text',
                          label_col=labels,
                          #batch_size_per_gpu=16,
                          #max_seq_length=512,
                          multi_gpu=True,
                          multi_label=True,
                          model_type="bert")

In [66]:
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
import logging

logger = logging.getLogger()
device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

In [ ]:
device = torch.device('cuda')

# check if multiple GPUs are available
if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

In [69]:
learner = BertLearner.from_pretrained_model(databunch, 
                                            pretrained_path = 'bert-base-uncased', 
                                            metrics=metrics, 
                                            device=device_cuda, 
                                            logger = logger, 
                                            output_dir=OUTPUT_DIR,
                                            finetuned_wgts_path=None, 
                                            is_fp16=args['fp16'], 
                                            loss_scale=args['loss_scale'], 
                                            multi_gpu=True,  
                                            multi_label=True
                                           )


AttributeError: 'BertDataBunch' object has no attribute 'model_type'

In [58]:
databunch.model_type

AttributeError: 'BertDataBunch' object has no attribute 'model_type'